In [1]:
"""purpose - create a persistent DuckDB database and store data from a couple datasets (i.e., as if they had been imported from CSV flatfiles) in it, each as their own table"""
from contextlib import contextmanager
import pathlib as pl
import sys
import warnings

import appdirs
import duckdb
import pandas as pd
from pydataset import data
from IPython.display import Markdown


# HACK the path for allow this notebook to import server's modules
THIS_DIR = globals()['_dh'][0]
REPO_ROOT = THIS_DIR.parent
SRC_DIR = (REPO_ROOT/'src').resolve()
#print(f"{THIS_DIR=}\n{REPO_ROOT=}\n{SRC_DIR=}")
sys.path.append(str(SRC_DIR))

from backend.sql.base import DUCKDB #<-- this import statement will implicitly create the DB if it doesn't exist
from backend.sql.duck import DuckDBMonitorMiddleware


data_dir = pl.Path(appdirs.site_data_dir("DemoApp", "JB"))
data_dir.mkdir(parents=True, exist_ok=True)

@contextmanager
def ignore_warnings():
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        yield

display(DUCKDB.PATH)

WindowsPath('C:/ProgramData/JB/DemoApp/dash.duckdb')

In [2]:
DUCKDB.init(wipe=True)
DUCKDB.init()
# DUCKDB.pseudo_ingest(
#     table_name='fake_table',
#     table_description='A fake table I made up to test pseudo-ingestion.',
#     table_type='system',
#     owned_by='jbechen'
# )


In [3]:
df1 = DuckDBMonitorMiddleware.get_dataframe("SELECT * FROM datasets.iris;")
#display(df1)
#df1['Species'] = df1['Species'].astype('category')
#display(f"{isinstance(df1['Species'].dtype, pd.CategoricalDtype)=}")

with ignore_warnings(), duckdb.connect(DUCKDB.PATH) as conn:
    df2 = pd.read_sql("SELECT * FROM administrative.table_catalog;", conn)
display(df2)

,table_name,table_description,table_type,owned_by,created,updates,updated,hits,last_hit,is_pseudo_table,etc
0,iris,"A 1936 dataset prepared by statistician Ronald Fisher, contains multivariate data desc...",system,unknown,2025-08-26 02:35:43.818,1,2025-08-26 02:35:43.908,1,2025-08-26 02:35:43.957,False,{}
1,economics,This dataset was produced from US economic time series data available from the Federal...,system,unknown,2025-08-26 02:35:43.821,1,2025-08-26 02:35:43.912,0,2025-08-26 02:35:43.821,False,{}
2,octoprint,A sensor stream from my personal Octoprint 3D printer server via HomeAssistant...,system,jbechen,2025-08-26 02:35:43.827,1,2025-08-26 02:35:43.918,0,2025-08-26 02:35:43.827,False,{}
3,pg_foo,A silly dummy DB I created to test the Postgres pseudo-ingestion path.,system,jbechen,2025-08-26 02:35:43.829,0,2025-08-26 02:35:43.829,0,2025-08-26 02:35:43.829,True,"{""cluster"":""nucleus.home.arpa"",""database"":""foo"",""sql"":""SELECT * FROM my_table;""}"
